# Problem 1 (20 pts)

In the case when you sample a sufficiently large number of normally distributed random variables $\{\eta_i\}_1^N$ with a given covariation function $k$ (using, for example, Cholesky decomposition of the covariance matrix or a square root of it), it may turn out that a smaller number of i.i.d random variables $\{\xi_i\}_1^m$ is sufficient to achieve a given accuracy. (Draw an analogy with the low-rank decomposition of the matrix, and the truncated KL expansion.)

Let the covariance of the GP $f$ with zero mean reads $$k(x,y)=\exp\left(-|x-y|^2/(2\ell^2)\right)$$ with $\ell=0.7$, and let we have $N=51$ points $\{x_i\}_{i=1}^{51}$
uniformly covering the interval $[-5,\,5]$ (so $x_i=-5+(i-1)/5)$. Consider a vector of r.v. $\eta$ with components $\eta_i=f(x_i)$, so it has a covariance matrix $K=k(\{x_i\},\{x_i\})$.
 - **(10 pts)** Find a linear combination $\theta=\mathbf v\cdot\mathbf \eta$ of r.v.  which is "almost" not random ($\sigma(\theta)\approx0$, where $\sigma$ is standard deviation). Here $\mathbf v\in\mathbb R^N$ is a deterministic vector of the unit length, $\|\mathbf v\|_2=1$. Try to find the best vector $\mathbf v$ in this sense.
 - **(10 pts)** Write a Python function that samples this $51$ r.v.  on the base of $m=10$ i.i.d. r.v. $\{\xi_i\}_{i=1}^m$ with the least possible error. 

In [6]:
import numpy as np
import scipy.spatial as SP

N = 51
ell = 0.7
x = np.linspace(-5, 5, N)


def kernel(a, b):
    sqdist = SP.distance.cdist(a, b, 'sqeuclidean')
    return np.exp(-.5 * sqdist/(ell**2))


def sample_K(K, N_samples=int(1e5)):
    N = len(K)
    L = np.linalg.cholesky(K + 1e-12*np.eye(N))
    xi = np.random.normal(size=(N, N_samples))
    eta = L.dot(xi)
    return eta


K = kernel(x[:, None], x[:, None])
np.random.seed(42)
smpl = sample_K(K)

# Problem 1a
# !!! Find this vector !!!
v = np.ones(N)
v /= np.linalg.norm(v, 2)

print("θ = {}...".format(v.dot(smpl)[:10]))
std = np.std(v.dot(smpl))
threshold = 2e-06
print("\n{0:.5g} {2} {1}".format(std, threshold, '<' if std < threshold else '?<?') )  # must be almost zero

θ = [ 1.8496188382 -2.3121479511  0.6349899156  2.4425555418  0.6682798465 -2.8134947731  1.5934918463  3.3011856958
  8.2929929663 -0.1606794129]...

2.8751 ?<? 2e-06


In [2]:
# Problem 1b
# Write this function
def sample_K_m(K, m, N_samples=int(1e5)):
    xi = np.random.normal(size=(m, N_samples))  # Only m, not N!
    # ...
    # ...
    # return eta


m = 10
#smpl = sample_K_m(K, m)


def empirical_cov(smpl):
    N, N_samples = smpl.shape
    res = np.zeros((N, N))
    for i in smpl.T:
        res += np.multiply.outer(i, i)

    return res/(N_samples-1)


K_emp = empirical_cov(smpl)
delta = np.max(np.abs((K - K_emp)))
print(delta)  # must be small

0.008134245251439354


# Problem 2 (60 pts)


Consider an stochastic PDE$\def\intrvl{[-2,\,1]}$
$$
\left\{
\begin{aligned}
\frac\partial{\partial t}u(x,t,\xi)&=
\frac\partial{\partial x}\left(k(x,\xi)\frac\partial{\partial x}u(t,x,\xi)\right)+1,&x&\in\intrvl,\;\;t\in[0,\,T],\\
u(t,-2,\xi)&=u(t,1,\xi)=0,&t&\in[0,\,T],\\
u(0,x,\xi)&=0,&x&\in\intrvl
\end{aligned}
\right..
$$
Here $k(x,\xi)$ is a random function with the mean $\mu=15$ and
the covariance function 
$$
\text{cov}\bigl(k(x,\xi),\,k(y,\xi)\bigr)=\frac32\exp\bigl(-0.3\left|x-y\right|\bigr).
$$

- **(5 pts)** Use Karhunen–Loève expansion to model the field $k$.
- **(30 pts)** Use gPC to represent the unknown value $u$,
solve the corresponding equation on the coefficients of the expansion and find the mean
and the variance of the solution $u$ at time $t=1$ as functions of $x$.
- **(15 pts)** Additionally, use Monte-Carlo method to model this mean and variance at the same time $t=1$.
- **(10 pts)** Compare  the results by plotting them on a single chart, find the optimal approximation parameters (the number of terms in the KL decomposition, the maximum degree of the polynomial in the gPC decomposition).

*Hint: you can use the analytical expressions for eigenfunction and eigenvalues and the code from Lecture 6 and use the code for multivariate gPC approximation form Lecture 7.*